In [3]:
import sympy
from symcircuit.symbolic_solver.domains import ExperimentResult

# Loading the Results of VLSI CMMF Exploration

In [6]:
results = ExperimentResult("VLSI_CMMF_POST_EXPERIMENT")
dir_of_results = results.find_results_file("Runs/VLSI_CMMF_Automated_NA")
results = results.load(f"{dir_of_results[0]}/results.pkl")

In [ ]:
results.classifications_dict.keys()

In [ ]:
classifications_of_intrest = results.classifications_dict["Z1_Z2_Z3_Z4_Z5_Z6"]
len(classifications_of_intrest), classifications_of_intrest[0]

In [13]:
df = results.flatten_classifications() # This way we can access all the classsifcation into a pandas df

# Examining the Outstanding Filters 

In [ ]:
classifications_of_intrest = [classification 
                              for classification in classifications_of_intrest 
                              if (not classification.valid) or (not classification.fType in ["BP", "BS", "LP", "HP", "GE"] )]
fTypes = set(classification.fType for classification in classifications_of_intrest)
len(classifications_of_intrest), fTypes

In [ ]:
classifications_of_intrest[0:10]

In [ ]:
single_classification = classifications_of_intrest[0]
# Since the original code classified the transfer functions by whether they fit to into a BiQuad filter, we classify them based on a different spec
single_classification.filterOrder, single_classification.fType, single_classification.parameters,  single_classification.zCombo, single_classification.valid

In [ ]:
count = {
    "CONST" : 0
}
s = sympy.symbols("s")

for i, classification in enumerate(classifications_of_intrest):
    tf = classification.transferFunc

    if not (s in tf.free_symbols): # TF id NOT a function of S - Pure amplifier/level-shifter
        ftype = "CONST"
        count[ftype] += 1
        classification.fType = "CONST"
        classification.filterOrder = "Zero"

    if classification.fType == "INVALID-WZ":
        # print(i, classification)
        pass

count

In [ ]:
classifications_of_intrest[32].transferFunc.simplify()

In [ ]:
params = classifications_of_intrest[32].parameters
params["Wz"].simplify() #, params["Wz"], params["wo"]

In [ ]:
params["Q"].simplify()

In [ ]:
sympy.ask()

In [ ]:
from sympy import symbols, ask, Q

# Declare symbols with assumptions
C1, C2, C3, C4, C5 = symbols('C1 C2 C3 C4 C5', positive=True)
s = symbols('s')  # 's' has no assumptions

# Define the expression
expr = (C1*C2 + C1*C3 - C1*C4 + C1*C5 + C2*C3)

# Check if the expression is positive
is_positive = ask(Q.positive(expr))

# Print the result
print("Is the expression positive?", is_positive)


In [ ]:
from sympy import symbols, ask, Q

# Declare symbols with assumptions
C1, C2, C3, C4, C5 = symbols('C1 C2 C3 C4 C5', positive=True)
s = symbols('s')  # 's' is unconstrained

# Define the expression
expr = (C1 * (C2 + C3 + C4 + C5) + C2 * C3)/C2
# Check if the expression could be positive (not strictly negative)
could_be_positive = not ask(Q.negative(expr))

# Print the result
print("Could the expression be positive (not strictly negative)?", could_be_positive)


In [ ]:
from sympy import symbols, Poly, Q, ask, solveset, S, simplify

def is_symbolic_stable(transfer_function, assumptions={}):
    """
    Checks if a transfer function with symbolic coefficients is stable.

    Args:
        transfer_function: A Sympy expression representing the transfer function.
        assumptions: A dictionary of assumptions for symbolic variables (e.g., {R1: Q.positive, C1: Q.positive}).

    Returns:
        True if the transfer function is stable (all poles have strictly negative real parts),
        False if any pole is in the right-hand plane,
        None if stability cannot be determined for some poles.
    """
    # Apply assumptions to all symbols
    for var, assumption in assumptions.items():
        var._assumptions.update({assumption: True})
    
    # Get the denominator of the transfer function
    denominator = transfer_function.as_numer_denom()[1]
    
    # Solve for poles (roots of the denominator)
    s = symbols('s')
    poles = solveset(denominator, s, domain=S.Complexes)
    
    # Check each pole
    for pole in poles:
        print(pole)
        # Simplify the pole and extract its real part
        real_part = simplify(pole.as_real_imag()[0])
        
        # Check if the real part is strictly negative
        is_negative = ask(Q.negative(real_part))
        if is_negative is False:  # Pole is in the right-hand plane or on the imaginary axis
            return False
        elif is_negative is None:  # Sympy can't determine
            return None
    
    # If all poles are strictly negative
    return True


# Example: Define symbolic components
R1, R2, R3 = symbols('R1 R2 R3', positive=True)  # Resistors
L1 = symbols('L1', positive=True)  # Inductor
C1, C2 = symbols('C1 C2', positive=True)  # Capacitors
s = symbols('s')

# Example transfer function with symbolic coefficients
G = (s + R1) / (L1 * s**2 + (R2 + R3) * s + 1 / C1)

# Define assumptions explicitly (optional, but helpful for ambiguous cases)
assumptions = {
    R1: Q.positive,
    R2: Q.positive,
    R3: Q.positive,
    L1: Q.positive,
    C1: Q.positive
}

# Check if the transfer function is stable
stability = is_symbolic_stable(G, assumptions)
print("Is the transfer function stable?", stability)


In [ ]:
expr = (s* R2/C2 + L1)*(s**2+ 3* R1/C2*s + L1)
expr = expr.expand()
expr.factor()

In [ ]:
from sympy import symbols, factor, Poly, simplify, solveset, S

def decompose_and_inspect(transfer_function, assumptions={}):
    """
    Decomposes the denominator of a transfer function into irreducible factors
    and inspects each factor for its properties (e.g., roots).

    Args:
        transfer_function: A Sympy expression representing the transfer function.
        assumptions: A dictionary of assumptions for symbolic variables (e.g., {R1: Q.positive, C1: Q.positive}).

    Returns:
        A dictionary where each key is a factor, and the value contains its roots and properties.
    """
    # Apply assumptions to all symbols
    for var, assumption in assumptions.items():
        var._assumptions.update({assumption: True})
    
    # Extract the denominator
    denominator = transfer_function.as_numer_denom()[1]
    
    # Factorize the denominator
    factors = factor(denominator)
    
    # Inspect each factor
    s = symbols('s')
    results = {}
    if factors.is_Mul:  # If there are multiple factors
        factors = factors.as_ordered_factors()
    else:
        factors = [factors]
    
    for factor_expr in factors:
        # Find the roots of this factor
        roots = solveset(factor_expr, s, domain=S.Complexes)
        
        # Analyze each root (e.g., check if it's strictly negative)
        roots_info = []
        for root in roots:
            real_part = simplify(root.as_real_imag()[0])
            roots_info.append({
                "root": root,
                "real_part": real_part
            })
        
        # Store results for this factor
        results[str(factor_expr)] = {
            "factor": factor_expr,
            "roots": roots_info,
            "degree": Poly(factor_expr).degree()
        }
    
    return results

# Example: Define symbolic components
R1, R2, R3 = symbols('R1 R2 R3', positive=True)  # Resistors
L1 = symbols('L1', positive=True)  # Inductor
C1, C2 = symbols('C1 C2', positive=True)  # Capacitors
s = symbols('s')

# Example transfer function
G = (s + R1) / (L1 * s**3 + (R2 + R3) * s**2 + 1 / C1 * s + 1 / C2)

# Decompose and inspect the factors
results = decompose_and_inspect(G)
for factor, info in results.items():
    print(f"Factor: {info['factor']}")
    print(f"Degree: {info['degree']}")
    print("Roots:")
    for root_info in info["roots"]:
        print(f"  - Root: {root_info['root']}, Real part: {root_info['real_part']}")
    print()


In [ ]:
sympy.factor()